# Jupyter Notebook UI to analyze baseline data from tap-habituation experiments!

### Beginner Essentials:
1. Shift-Enter to run each cell. After you run, you should see an output "done step #". If not, an error has occured
2. When inputting your own code/revising the code, make sure you close all your quotation marks '' and brackets (), [], {}.
3. Don't leave any commas (,) hanging! (make sure an object always follows a comma. If there is nothing after a comma, remove the comma!
4. Learning to code? Each line of code is annotated to help you understand how this code works!

**Run all cells/steps sequentially, even the ones that do not need input**

## Step-by-Step Analysis of the Jupyter Notebook

| Step | Purpose | Key Actions |
|------|---------|-------------|
| **1. Import Packages** | Load required Python libraries for data analysis | Imports `pandas`, `numpy`, `matplotlib`, etc. | 
| **2. Pick Filepath** | Select the folder containing experimental data files (.dat or .trv) | Input required: Uses `FileChooser` widget to select directory | 
| **3. User-Defined Variables** | Set experiment parameters | Defines: `bin`  | 
| **4. Construct Filelist** | Find all files in selected folder | Sets working directory and scans `folder_path` using; Displays no. of `.trv` files found in the folder |
| **5. Process Data Function** | Define functions to load, clean, and analyze raw data | - `ProcessData()`: Loads files, calculates metrics (reversal probability, speed) |
| **6.1 Process Data** | Apply processing to all strains| - Checks `filelist` for unique strain names (e.g., "N2") <br>- Runs `ProcessData()` for each strain | 
| **7. Grouping & Naming** | Combine data from all strains | - Concatenates DataFrames<br>- Assigns dataset names (e.g., "N2") | 
| **Output CSV** | Save processed data | Exports `Baseline_data` to CSV |

### Key Notes:
- User Input Required: Steps 2 (file selection), 3 (parameters), 6.1 (strain verification)
- Output: Final CSV contains all analyzed tap response data

# 1. Importing Packages Required (No input required, just run)

In [129]:
import pandas as pd #<- package used to import and organize data
import numpy as np #<- package used to import and organize data
import seaborn as sns #<- package used to plot graphs
from matplotlib import pyplot as plt #<- package used to plot graphs
import os #<- package used to work with system filepaths
from ipywidgets import widgets #<- widget tool to generate button
from IPython.display import display #<- displays button
from ipyfilechooser import FileChooser
# from tkinter import Tk, filedialog #<- Tkinter is a GUI package
from tqdm.notebook import tqdm
# import dask.dataframe as dd
# import pingouin as pg
pd.set_option('display.max_columns', 50)
print("done step 1")

done step 1


## 2. Pick filepath (just run and click button from output)

Run the following cell and click the button 'Select Folder' to pick a filepath.

**Important: Later on, this script uses the total file path for each file to import and group data. That means if your folder has whatever your strain is named, the script will not work.**

(ex. if your folder has "N2" in it this script sees all files inside this folder as having the "N2" search key)

**An easy fix is to just rename your folder to something else (make your strains lower-case, or just have the date)**

In [2]:
starting_directory = '/Users'
chooser = FileChooser(starting_directory)
display(chooser)

FileChooser(path='/Users', filename='', title='', show_hidden=False, select_desc='Select', change_desc='Change…

In [130]:
print(chooser.selected_path)
folder_path=chooser.selected_path

/Users/gurmehak/Documents/RankinLab/Test_Datasets/PDScreen_TapHab_August15_2022


In [24]:
screens = ['PD_Screen', 'ASD_Screen', 'G-Proteins_Screen', 'Glia_Genes_Screen', 'Neuron_Genes_Screen']

screen_chooser = widgets.Select(options=screens, value=screens[0], description='Screen:')
display(screen_chooser)

Select(description='Screen:', options=('PD_Screen', 'ASD_Screen', 'G-Proteins_Screen', 'Glia_Genes_Screen', 'N…

In [131]:
Screen=screen_chooser.value

# 3. User Defined Variables (Add input here)

Here, we add some constants to help you blaze through this code.

3.1: Setting time bins


3.2: Setting view range for your graph
- Top, bottom = y axis view range
- left, right = x axis view range



In [132]:
# Setting 1s Bins
bins = np.linspace(0,1200,1201) # np.linspace(start, end, steps in between)
print(bins)


print("done step 3")

[0.000e+00 1.000e+00 2.000e+00 ... 1.198e+03 1.199e+03 1.200e+03]
done step 3


# 4. Construct filelist from folder path (No input required, just run)

In [133]:
os.chdir(folder_path) # setting your working directory so that your images will be saved here

filelist = list() # empty list
for root, dirs, files in os.walk(folder_path): # this for loop goes through your folder 
    for name in files:
        if name.endswith('.dat'): # and takes out all files with a .dat (file that contains your data)
            if "_" in name.split(".")[-2]:
                filepath = os.path.join(root, name) # Notes down the file path of each data file
                filelist.append(filepath) # saves it into the list

if not filelist:
    raise FileNotFoundError("No .dat files found in the selected folder!")
else:
    print(f"Number of .dat files to process: {len(filelist)}")
    # print(f"Example of first and last file saved: {filelist[0]}, {filelist[-1]}") 

print('done step 4')

Number of .dat files to process: 13
done step 4


# 5. Process Data Function (No input required, just run)

In [154]:
def ProcessData(strain, experiment_counter): 
    """
    Filters and processes .dat files matching the given strain.

    Parameters: 
        strain (str): keyword to match in the files

    Returns:
        dict: N (Plate number) and Dataframe with required columns 
              ("time", "dura", "dist", "prob", "speed", "plate", "Date",
              "Plate_id", "Screen")

    """
    strain_filelist = [x for x in filelist if strain in x] # Goes through the list and filters for keyword
    Strain_N = len(strain_filelist) # Finds the number of plates per strain
    if Strain_N == 0:
        raise AssertionError ('{} is not a good identifier'.format(strain))
    else:
        pass
        print(f'Strain {strain}')
        print(f'Number of plates: {Strain_N}') 
        
        # visiting files in this strain
        strain_filelist = [file for file in filelist if strain in file]
        df_list=[]
        for i, file in enumerate(strain_filelist):
            if file.split('/')[-1].startswith('._'):
                pass
            else:
                try:
                    print(f"File: {file}")
                    df= pd.read_csv(file, sep=' ', header = None, encoding_errors='ignore')
                    df['Plate_id'] = file.split('/')[-1].split('_')[-1].split('.')[0]
                    df['Date'] = file.split('/')[-2].split('_')[0]
                    df['Screen'] = file.split('/')[-4]
                    df['Experiment'] = experiment_counter
                    experiment_counter = 1+experiment_counter
                    df_list.append(df)
                except:
                    print(f"error in file {file}")
                    pass
        DF_Total = pd.concat(df_list, ignore_index = True)
        DF_Total = DF_Total.rename( 
                    {0:'Time',
                    1:'n',
                    2:'Number',
                    3:'Instantaneous Speed',
                    4:'Interval Speed',
                    5:'Bias',
                    6:'Tap',
                    7:'Puff',
                    8:'x',
                    9:'y',
                    10:'Morphwidth',
                    11:'Midline',
                    12:'Area',
                    13:'Angular Speed',
                    14:'Aspect Ratio',
                    15:'Kink',
                    16:'Curve',
                    17:'Crab',
                    18:'Pathlength'}, axis=1)
        
        # check function here for NaN Columns
        DF_Total['plate'] = 0

        print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

    return{
            'N': Strain_N,
            'Confirm':DF_Total,
            'experiment_counter': experiment_counter
            # 'Final': DF_Final
    }


print('done step 5')

done step 5


# 6.1 Process Data

Create a dictionary `StrainNames` that contains all the genotype/strain names from each file path

In [155]:
genotype=[]
for f in filelist:
    genotype.append(f.split('/')[-3])

genotypes=np.unique(genotype).tolist()

if Screen =="Neuron_Genes_Screen":
    genotypes.insert(0, genotypes.pop(genotypes.index("N2_XJ1")))
    genotypes.insert(0, genotypes.pop(genotypes.index("N2_N2")))
else:
    genotypes.insert(0, genotypes.pop(genotypes.index("N2")))

nstrains = list(range(1, len(genotypes) + 1))
StrainNames = {nstrains[i]: genotypes[i] for i in range(len(nstrains))}

print(f"Number of genotypes/strains in the experiment: {len(genotypes)}")

# Display the first 5 Strain names in the experiment
for k in list(StrainNames)[:5]:
    print(f"{k}: {StrainNames[k]}")


print("done step 6.1")

# <---------------- Test element to use for dictionary buidling -------------------
# s = '/Users/Joseph/Desktop/OnFoodOffFoodTest/N2_OnFood/20220401_163048/N2_10x1_n96h20C_360sA0401_ka.00065.dat'
# slist=s.split('/')[5]
# print(slist)
# print(list(range(1,5+1)))

Number of genotypes/strains in the experiment: 3
1: N2
2: hipr-1_ok1081
3: hipr-1_tm10120
done step 6.1


# 6.2 Process Data (just run this cell)

Pass each strain through `ProcessData()` function 

In [158]:
DataLists = [0] # generates empty list at index 0 because we want indexing to start at 1 
                # when I say #1, I want the first point, not the second point

experiment_counter = 1

# the loop below goes through the dictionary in step 6.1 and processes data
# and appends all data into a list of dataframes
for s in tqdm(StrainNames.values()): 
    if not s == '':
        result = ProcessData(s, experiment_counter)
        DataLists.append(result['Confirm'])
        experiment_counter = result['experiment_counter'] 

print('done step 6.2')

  0%|          | 0/3 [00:00<?, ?it/s]

Strain N2
Number of plates: 5
File: /Users/gurmehak/Documents/RankinLab/Test_Datasets/PDScreen_TapHab_August15_2022/N2/20220815_101538/N2_10x2_f72h20C_600s31x10s10s_B0811ab.dat
File: /Users/gurmehak/Documents/RankinLab/Test_Datasets/PDScreen_TapHab_August15_2022/N2/20220815_102652/N2_10x2_f96h20C_600s31x10s10s_A0811aa.dat
File: /Users/gurmehak/Documents/RankinLab/Test_Datasets/PDScreen_TapHab_August15_2022/N2/20220815_122801/N2_10x2_f96h20C_600s31x10s10s_A0811ad.dat
File: /Users/gurmehak/Documents/RankinLab/Test_Datasets/PDScreen_TapHab_August15_2022/N2/20220815_121502/N2_10x2_f72h20C_600s31x10s10s_B0811ae.dat
File: /Users/gurmehak/Documents/RankinLab/Test_Datasets/PDScreen_TapHab_August15_2022/N2/20220815_103433/N2_10x2_f72h20C_600s31x10s10s_C0811ac.dat
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Strain hipr-1_ok1081
Number of plates: 4
File: /Users/gurmehak/

# Convert float64 data to float32 to reduce memory load (can also convert to 16 if needed)

For plain english:

float16 = 4 decimal points

float32 = 8 decimal points

float64 = 16 decimal points

more decimal points = more data/memory that computer has to keep track of

In [40]:
# commented out this section in case memory load needs to be reduced

# for n in tqdm(DataLists[1:]:
#     print(n)
#     TestData=n
#     TestData[TestData.select_dtypes(np.float64).columns]=TestData.select_dtypes(np.float64).astype(np.float16)
    

# 7. Grouping Data and Naming

This step takes all the individual strain data (processed in Step 6) and combines them into single dataframe, filters for time window 490s - 590s, drops unwanted columns. 

The final processed data `Baseline_data` is ready for analysis.

In [273]:
Baseline_data=pd.concat(df.assign(dataset=StrainNames.get(i+1)) for i, df in enumerate(DataLists[1:]))

Baseline_data[['Gene', 'Allele']] = Baseline_data['dataset'].str.split('_', n=1, expand=True)

Baseline_data['Allele'] = Baseline_data['Allele'].fillna('N2')

Baseline_data = Baseline_data.drop(columns=["plate", "Tap", "Puff", "x","y", "Experiment"]).dropna().reset_index(drop=True)

Baseline_data['Screen']=Screen

Baseline_data = Baseline_data[((Baseline_data.Time<=590)&(Baseline_data.Time >=490))] 

Baseline_data.head()

,Time,n,Number,Instantaneous Speed,Interval Speed,Bias,Morphwidth,Midline,Area,Angular Speed,Aspect Ratio,Kink,Curve,Crab,Pathlength,Plate_id,Date,Screen,dataset,Gene,Allele
9775,490.016,14,12,0.0823,0.1195,0.25,0.1078,1.0908,0.142641,6.1,0.225,31.1,25.0,0.0066,11.893,B0811ab,20220815,PD_Screen,N2,N2,N2
9776,490.056,14,12,0.0736,0.1024,0.25,0.1059,1.0880,0.140150,5.3,0.219,30.9,24.7,0.0064,11.896,B0811ab,20220815,PD_Screen,N2,N2,N2
9777,490.103,14,12,0.0784,0.1024,0.25,0.1050,1.0914,0.138935,5.2,0.219,31.0,24.6,0.0057,11.898,B0811ab,20220815,PD_Screen,N2,N2,N2
9778,490.144,14,12,0.0970,0.1118,0.25,0.1054,1.0935,0.140575,5.8,0.218,30.6,24.5,0.0103,11.901,B0811ab,20220815,PD_Screen,N2,N2,N2
9779,490.186,14,12,0.0994,0.1197,0.25,0.1110,1.1026,0.146894,5.6,0.227,31.4,24.4,0.0097,11.904,B0811ab,20220815,PD_Screen,N2,N2,N2


In [43]:
Baseline_data.shape

(30487, 21)

## Creating Post Stimulus Data 

In [268]:
# similar filters as baseline data

Post_stimulus_data_pre=pd.concat(df.assign(dataset=StrainNames.get(i+1)) for i,df in enumerate(DataLists[1:]))

Post_stimulus_data_pre[['Gene', 'Allele']] = Post_stimulus_data_pre['dataset'].str.split('_', n=1, expand=True)

Post_stimulus_data_pre['Allele'] = Post_stimulus_data_pre['Allele'].fillna('N2')

Post_stimulus_data_pre = Post_stimulus_data_pre.drop(columns=["plate", "Puff", "x","y"]).dropna().reset_index(drop=True)

Post_stimulus_data_pre['Screen']=Screen

Post_stimulus_data_pre = Post_stimulus_data_pre[((Post_stimulus_data_pre.Time>599))]

Post_stimulus_data_pre['Time'] = round(Post_stimulus_data_pre['Time']).astype('int')

In [ ]:
# Add continuous tap numbers from 1 to 31 for each experiment
# E.g., Experiment 1 has taps 1-31, Experiment 2 has taps 1-31 and so on..

Post_stimulus_data_pre['Tap_num'] = Post_stimulus_data_pre.groupby(['Experiment'])['Tap'].cumsum()

Post_stimulus_data_pre.head()

,Time,n,Number,Instantaneous Speed,Interval Speed,Bias,Tap,Morphwidth,Midline,Area,Angular Speed,Aspect Ratio,Kink,Curve,Crab,Pathlength,Plate_id,Date,Screen,Experiment,dataset,Gene,Allele,Tap_num
12094,599,16,14,0.0692,0.1084,0.429,0,0.1045,1.0889,0.139812,3.7,0.249,38.8,27.6,0.0067,6.614,B0811ab,20220815,PD_Screen,1,N2,N2,N2,0
12095,599,16,14,0.0885,0.1161,0.429,0,0.1046,1.0857,0.139187,4.8,0.247,37.1,27.4,0.0085,6.618,B0811ab,20220815,PD_Screen,1,N2,N2,N2,0
12096,599,16,14,0.0967,0.1244,0.429,0,0.1059,1.0914,0.141114,5.2,0.250,39.6,27.6,0.0092,6.622,B0811ab,20220815,PD_Screen,1,N2,N2,N2,0
12097,599,16,14,0.1135,0.1604,0.571,0,0.1103,1.1059,0.146321,5.1,0.253,38.1,27.5,0.0094,6.626,B0811ab,20220815,PD_Screen,1,N2,N2,N2,0
12098,599,16,14,0.1131,0.1630,0.571,0,0.1059,1.0967,0.141478,4.9,0.248,37.9,27.4,0.0111,6.632,B0811ab,20220815,PD_Screen,1,N2,N2,N2,0


In [ ]:
# Create windows from 7s to 9.5s post a tap ("Tap"=1) for each experiment
# and concatenate all these wondows into a single dataframe

Post_stimulus_data = []

for exp in Post_stimulus_data_pre['Experiment'].unique(): # loop through each experiment separately 
    df = Post_stimulus_data_pre[Post_stimulus_data_pre['Experiment'] == exp]  
    tap_times = df[df['Tap'] == 1]['Time']  # get times where tap occured

    for t in tap_times: 
        window = df[(df['Time'] >= t + 7) & (df['Time'] <= t + 9.5)]
        Post_stimulus_data.append(window)

Post_stimulus_data = pd.concat(Post_stimulus_data)

Post_stimulus_data.head()


,Time,n,Number,Instantaneous Speed,Interval Speed,Bias,Tap,Morphwidth,Midline,Area,Angular Speed,Aspect Ratio,Kink,Curve,Crab,Pathlength,Plate_id,Date,Screen,Experiment,dataset,Gene,Allele,Tap_num
12270,607,17,14,0.1634,0.1742,0.500,0,0.1153,1.0888,0.145435,7.5,0.380,56.0,39.2,0.0156,6.378,B0811ab,20220815,PD_Screen,1,N2,N2,N2,1
12271,607,17,14,0.1317,0.1498,0.500,0,0.1132,1.0722,0.142155,7.6,0.382,54.8,39.0,0.0156,6.387,B0811ab,20220815,PD_Screen,1,N2,N2,N2,1
12272,607,17,14,0.1287,0.1541,0.500,0,0.1182,1.0861,0.148143,6.3,0.380,57.8,38.8,0.0154,6.397,B0811ab,20220815,PD_Screen,1,N2,N2,N2,1
12273,607,17,14,0.1197,0.1546,0.571,0,0.1147,1.0684,0.143301,5.6,0.370,56.5,37.9,0.0164,6.406,B0811ab,20220815,PD_Screen,1,N2,N2,N2,1
12274,607,17,14,0.1149,0.1522,0.571,0,0.1112,1.0703,0.139916,6.9,0.375,54.5,37.7,0.0136,6.415,B0811ab,20220815,PD_Screen,1,N2,N2,N2,1


In [272]:
# Aggregate columns by "Experiment" + "Tap_num" by taking their means

Post_stimulus_data = Post_stimulus_data.groupby(['Experiment', 'Tap_num']).agg({
    'Time': 'min', # take minimum valu of time instead of mean
    'n': 'mean',
    'Number': 'mean',
    'Instantaneous Speed': 'mean',
    'Interval Speed' : 'mean',
    'Bias': 'mean',
    'Tap': 'mean',
    'Morphwidth': 'mean',
    'Midline': 'mean',
    'Area': 'mean',
    'Angular Speed': 'mean',
    'Aspect Ratio': 'mean',
    'Kink': 'mean',
    'Curve': 'mean',
    'Crab': 'mean',
    'Pathlength': 'mean'
})

Post_stimulus_data = Post_stimulus_data.reset_index()

Post_stimulus_data

,Experiment,Tap_num,Time,n,Number,Instantaneous Speed,Interval Speed,Bias,Tap,Morphwidth,Midline,Area,Angular Speed,Aspect Ratio,Kink,Curve,Crab,Pathlength
0,1,1,607,16.925000,14.000000,0.125645,0.135265,0.350025,0.0,0.112155,1.072315,0.141810,7.340000,0.344050,48.107500,36.130000,0.015625,6.499075
1,1,2,617,16.972973,14.891892,0.214500,0.144012,0.527905,0.0,0.111914,1.051191,0.137795,16.145946,0.315459,53.440541,32.040541,0.026011,6.607365
2,1,3,627,16.629032,13.000000,0.235965,0.107100,0.477129,0.0,0.109982,1.079510,0.140502,12.866129,0.293274,51.256452,30.583871,0.025710,7.168435
3,1,4,637,13.134328,12.000000,0.235875,0.126124,0.965970,0.0,0.105619,1.042824,0.133841,20.716418,0.333090,58.326866,31.059701,0.026849,8.156448
4,1,5,647,16.108108,12.000000,0.266666,0.124458,0.958500,0.0,0.103418,1.075350,0.137081,13.786486,0.281851,39.606757,29.005405,0.027261,9.523203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,13,27,867,24.480000,21.626667,0.150065,0.086068,0.895293,0.0,0.089453,0.974081,0.110356,6.172000,0.238000,40.257333,26.874667,0.011888,14.472000
398,13,28,877,23.120000,21.040000,0.148219,0.085596,0.856187,0.0,0.088217,0.962596,0.108167,4.645333,0.218173,35.217333,25.397333,0.009620,12.475093
399,13,29,887,23.391892,22.391892,0.129711,0.077372,0.782405,0.0,0.091149,0.934377,0.105980,5.264865,0.217757,35.762162,25.420270,0.009753,12.697027
400,13,30,897,21.547945,20.547945,0.136551,0.079448,0.855685,0.0,0.086688,0.952897,0.105771,4.691781,0.212603,37.989041,25.341096,0.009581,12.224658


In [24]:
print('done step 7')

done step 7


# Save dataframe as `.csv`

In [ ]:
Baseline_data.to_csv(f"{Baseline_data.Screen[0]}_baseline_output.csv")
print('saved Baseline data as .csv!')

saved as .csv!


In [ ]:
Post_stimulus_data.to_csv(f"{Post_stimulus_data.Screen[0]}_post_stimulus.csv")
print('saved Post stimulus data as .csv!')

# Done!